In [1]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline
import pandas as pd

In [2]:
finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')

nlp = pipeline("sentiment-analysis", model=finbert, tokenizer=tokenizer)

Device set to use mps:0


In [3]:
df = pd.read_csv('nvda.csv')
df


,Unnamed: 0.1,Unnamed: 0,title,datetime,source,link,sentiment_label,sentiment_score
0,0,0,"5 Top Stock Trades for Tuesday: NVDA, TSLA, GILD",11/25/2019 9:11:58 PM,InvestorPlace,/news/stocks/5-top-stock-trades-for-tuesday-nv...,Neutral,0.999999
1,1,1,Stock Market Today: Merger Monday; Tesla’s Cyb...,11/25/2019 9:26:20 PM,InvestorPlace,/news/stocks/stock-market-today-merger-monday-...,Neutral,0.999994
2,2,2,10 Fantastic Tech Gifts to Buy for $500 and Up,11/26/2019 11:00:01 AM,InvestorPlace,/news/stocks/10-fantastic-tech-gifts-to-buy-fo...,Positive,0.985829
3,3,3,"Shares Could Go Higher, but Nvidia Stock Is Ov...",11/26/2019 2:08:09 PM,InvestorPlace,/news/stocks/nvidia-stock-overvalued-higher-10...,Negative,0.991712
4,4,4,"Tuesday’s Vital Data: Nvidia, Bank of America ...",11/26/2019 2:22:29 PM,InvestorPlace,/news/stocks/tuesdays-vital-data-nvidia-bank-o...,Neutral,0.999958
...,...,...,...,...,...,...,...,...
8944,8944,8944,Quantum announces support for ‘NVIDIA GPUDirec...,12/18/2024 1:15:33 PM,TipRanks,/news/stocks/quantum-announces-support-for-nvi...,Positive,0.998504
8945,8945,8945,"4 stocks to watch on Wednesday: GIS, MU and more",12/18/2024 1:55:00 PM,Seeking Alpha,https://seekingalpha.com/news/4387654-4-stocks...,Neutral,0.999953
8946,8946,8946,Notable open interest changes for December 18th,12/18/2024 2:02:43 PM,TipRanks,/news/stocks/notable-open-interest-changes-for...,Neutral,0.999996
8947,8947,8947,Early notable gainers among liquid option name...,12/18/2024 2:55:13 PM,TipRanks,/news/stocks/early-notable-gainers-among-liqui...,Positive,0.940363


In [4]:
def calculate_sentiment_score(row):
    text = row['title']
    result = nlp(text)
    return result[0]['score']

def calculate_sentiment_label(row):
    text = row['title']
    result = nlp(text)
    return result[0]['label']

In [5]:
df['sentiment_score'] = df.apply(calculate_sentiment_score, axis=1)
df['sentiment_label'] = df.apply(calculate_sentiment_label, axis=1)

df

KeyboardInterrupt: 

In [ ]:
df.to_csv('/Users/abelsaj/Documents/Personal Coding Projects/Stock-Prediction-Model/nvda.csv')

In [ ]:
df['sentiment_label'].value_counts()

sentiment_label
Neutral     4298
Positive    3542
Negative    1109
Name: count, dtype: int64

In [54]:
df['sentiment_score'].describe()

count    8949.000000
mean        0.952052
std         0.108865
min         0.347309
25%         0.976779
50%         0.999348
75%         0.999986
max         1.000000
Name: sentiment_score, dtype: float64

In [ ]:
nvda_sentiment = pd.read_csv('nvda.csv')
nvda_sentiment = nvda_sentiment.drop(['source', 'link', 'Unnamed: 0', 'Unnamed: 0.1'], axis = 1)

In [ ]:
nvda_sentiment['datetime'] = pd.to_datetime(nvda_sentiment['datetime']).dt.date
nvda_sentiment

In [ ]:
aggregated = nvda_sentiment.groupby('datetime').agg({
    'sentiment_label' : lambda x : x.mode()[0],
    'sentiment_score' : 'mean'
}).reset_index()

aggregated = aggregated.sort_values(by='datetime')
aggregated.to_csv('nvda_sentiment_aggregated.csv', index = False)

In [ ]:
nvda_sentiment_aggregated = pd.read_csv('nvda_sentiment_aggregated.csv')
nvda_sentiment_aggregated.value_counts('sentiment_label')
nvda_sentiment_aggregated.rename(columns={'datetime': 'date'}, inplace=True)
nvda_sentiment_aggregated